In [1]:
from mplsoccer.statsbomb import Sbopen
from db_connection import get_db
from sql_schemas import Matches, Events, EventTypes, Teams, Competitions, PlayerPositions, Person
import pandas as pd
import numpy as np
import ipywidgets as widgets

statsbomb_api = Sbopen()
db = next(get_db())

In [2]:
from requests import Session


def insert_player_positions(db: Session, df_event):
    event_records = df_event[2].to_dict(orient="records")
    
    event_ids = {p["id"] for p in event_records}

    existing_positions = db.query(PlayerPositions.event_id).filter(PlayerPositions.event_id.in_(event_ids)).all()
    existing_event_ids = {pos.event_id for pos in existing_positions}

    new_positions = [
        PlayerPositions(
            match_id=p['match_id'], teammate=p['teammate'], x=p['x'], y=p['y'],
            event_id=p['id'], player_id=p['player_id'], position_id=p['position_id']
        )
        for p in event_records if p["id"] not in existing_event_ids
    ]

    # Bulk insert only new records
    if new_positions:
        db.bulk_save_objects(new_positions)
        db.commit()

In [ ]:
def insert_competitions(db: Session, df_competitions):
    competition_records = df_competitions.to_dict(orient="records")
    
    competition_ids = {p["competition_id"] for p in competition_records}

    existing_positions = db.query(Competitions.competition_id).filter(Competitions.competition_id.in_(competition_ids)).all()
    existing_competition_ids = {pos.competition_id for pos in existing_positions}
    new_competitions = [
        Competitions(
            competition_id=p['competition_id'],competition_name=p['competition_name'], season_id=p['season_id'],  season_name=p['season_name'],
            country_name=p['country_name'], competition_gender=p['competition_gender'],
            competition_youth=p['competition_youth'], competition_international=p['competition_international']
        )
        for p in competition_records if p['competition_id'] not in existing_competition_ids
    ]

    if new_competitions:
        db.bulk_save_objects(new_competitions)
        db.commit()


In [ ]:
def insert_event_data(db: Session, df_event):
    df = df_event[0]
    df = df.fillna(-1)
    event_records = df.to_dict(orient="records")
    
    event_ids = {p["id"] for p in event_records}
    existing_events = db.query(Events).filter(Events.id.in_(event_ids)).all()
    existing_event_ids = {pos.id for pos in existing_events}

    new_events = [
        Events(
            id=p['id'], match_id=p['match_id'], x=p['x'], y=p['y'],
            type_id=p['type_id'], player_id=p['player_id'], position_id=p['position_id'],
             end_x=p['end_x'], end_y=p['end_y'], period=p['period'], timestamp=p['timestamp']
        )
        for p in event_records if p['id'] not in existing_event_ids
    ]
    
    if new_events:
        db.bulk_save_objects(new_events)
    event_types = df.loc[:,['type_id','type_name']].drop_duplicates().to_dict(orient="records")


    event_type_ids = {p["type_id"] for p in event_types}
    existing_event_types = db.query(EventTypes).filter(EventTypes.id.in_(event_type_ids)).all()
    existing_event_type_ids = {pos.id for pos in existing_event_types}

    new_event_types = [
        EventTypes(
            id=p['type_id'], name=p['type_name']
        )
        for p in event_types if p["type_id"] not in existing_event_type_ids
    ]
    if new_event_types:
        db.bulk_save_objects(new_event_types)

    players = df.loc[:,['player_id','player_name']].drop_duplicates().to_dict(orient="records")
    player_ids = {p["player_id"] for p in players}
    existing_players = db.query(Person).filter(Person.id.in_(player_ids)).all()
    existing_player_ids = {pos.id for pos in existing_players}
    existing_player_ids.add(-1)
    new_players = [
        Person(
            id=p['player_id'], name=p['player_name']
        )
        for p in players if p["player_id"] not in existing_player_ids
    ]
    if new_players:
        db.bulk_save_objects(new_players)

In [ ]:
competition = statsbomb_api.competition()

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Loading:',
    bar_style='',
    style={'bar_color': 'blue'},
    orientation='horizontal'
)
display(progress)

competitions = competition[~competition['match_available_360'].isna()]
insert_competitions(db, competitions)
for index, row in competitions.iterrows():
    matches = statsbomb_api.match(competition_id=row['competition_id'], season_id=row['season_id'])
    matches = matches[matches['match_status_360'] =='available']
    matches = matches.fillna(-1)
    progress.description = f"{row['competition_name']}: "
    progress.value = 0
    progress.max = len(matches)

    for matchindex, match in matches.iterrows():
        progress.value+=1
        #player_pos, events = a.frame(match_id=match['match_id'])
        db_match = Matches(id=match['match_id'], 
                           match_date=match['match_date'], 
                           away_score=match['away_score'], 
                           home_score=match['home_score'],
                           home_team_id=match['home_team_id'],
                           home_manager_id=match['home_team_managers_id'],
                           away_team_id=match['away_team_id'],
                           away_manager_id=match['away_team_managers_id'],
                           competition_id = match['competition_id'],
                           match_week=match['match_week'],
                           season_id=match['season_id'],
                           referee_id=match['referee_id'],
                           stadium_id=match['stadium_id'],
                           )
        qry_object = db.query(Matches).where(Matches.id == db_match.id)
        if qry_object.first() is None:
            db.add(db_match)
        db_home_team=Teams(id=match['home_team_id'], name=match['home_team_name'])
        qry_object = db.query(Teams).where(Teams.id == db_home_team.id)
        if qry_object.first() is None:
            db.add(db_home_team)
        db_away_team = Teams(id=match['away_team_id'], name=match['away_team_name'])
        qry_object = db.query(Teams).where(Teams.id == db_away_team.id)
        if qry_object.first() is None:
            db.add(db_away_team)
        db.commit()
        df_event = statsbomb_api.event(match['match_id'])
        insert_event_data(db=db,df_event=df_event)
        insert_player_positions(db=db,df_event=df_event)

db.close()

IntProgress(value=0, description='Loading:', max=10, style=ProgressStyle(bar_color='blue'))

set()


C:\Users\juras\AppData\Local\Temp\ipykernel_20472\1270744928.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matches = matches.fillna(-1)


[{'player_id': -1.0, 'player_name': -1}, {'player_id': 34870.0, 'player_name': 'Nick Woltemade'}, {'player_id': 12299.0, 'player_name': 'Marvin Ducksch'}, {'player_id': 31100.0, 'player_name': 'Christian Groß'}, {'player_id': 51769.0, 'player_name': 'Julián Malatini'}, {'player_id': 33401.0, 'player_name': 'Amine Adli'}, {'player_id': 8576.0, 'player_name': 'Mitchell Weiser'}, {'player_id': 30606.0, 'player_name': 'Edmond Fayçal Tapsoba'}, {'player_id': 8221.0, 'player_name': 'Jonathan Tah'}, {'player_id': 9195.0, 'player_name': 'Robert Andrich'}, {'player_id': 28573.0, 'player_name': 'Felix Agu'}, {'player_id': 41411.0, 'player_name': 'Nathan Tella'}, {'player_id': 8667.0, 'player_name': 'Lukáš Hrádecký'}, {'player_id': 27133.0, 'player_name': 'Odilon Kossonou'}, {'player_id': 8784.0, 'player_name': 'Romano Schmid'}, {'player_id': 3500.0, 'player_name': 'Granit Xhaka'}, {'player_id': 8826.0, 'player_name': 'Leonardo Bittencourt'}, {'player_id': 32289.0, 'player_name': 'Victor Okoh Bon